In [4]:
import webdnn

In [3]:
"""
Keras model converter
"""

import argparse
import importlib.util
import inspect
import os
import sys
sys.setrecursionlimit(10000)  # workaround for deep copying large graph
import traceback
from os import path

import keras

from webdnn.backend import generate_descriptor
from webdnn.frontend.keras import KerasConverter
from webdnn.graph import traverse
from webdnn.graph.placeholder import Placeholder
from webdnn.graph.shape import Shape
from webdnn.graph.traverse import dump_dot
from webdnn.util import flags, console

In [8]:
input_shapes = [Shape.parse('(1, 512, 512, 3)')[0]]
model_path = 'full_boneage_model.h5'
model = keras.models.load_model(model_path, 
                                custom_objects=[], compile=False)

In [9]:
model.build(input_shape=None)
converter = KerasConverter(batch_size=input_shapes[0][0])
graph = converter.convert(model)
traverse.dump(graph)

/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/webdnn/util/console.py:30: Warning: [KerasConverter] keras.layers.AveragePooling computes average by dividing number of valid elements in window (without padding element), but WebDNN divides it by the number of elements including padding element, so different result will be generated on the edge.
  warnings.warn(message, category)
/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/webdnn/util/console.py:30: Warning: [KerasConverter] omitting dropout
  warnings.warn(message, category)


In [11]:
for graph_input, input_shape in zip(graph.inputs, input_shapes):
    for p1, p2 in zip(graph_input.shape, input_shape):
        if not Placeholder.check_resolved(p1) and Placeholder.check_resolved(p2):
            p1.value = Placeholder.force_int(p2)

        elif Placeholder.check_resolved(p1) and not Placeholder.check_resolved(p2):
            raise ValueError(f'Shape mismatch: expected:{input_shape}, real:{graph_input.shape}, {p1} != {p2}')

        elif Placeholder.check_resolved(p1) and Placeholder.check_resolved(p2):
            assert p1 == p2, f'Shape mismatch: expected:{input_shape}, real:{graph_input.shape}, {p1} != {p2}'

output_dir = path.join(path.dirname(model_path), "webdnn_graph_descriptor")
os.makedirs(output_dir, exist_ok=True)

ir_dot_path = path.join(output_dir, "ir.dot")
with open(ir_dot_path, "w") as f:
    f.write(dump_dot(graph))
console.stderr(f"IR graph can be visualized with graphviz command: 'dot {ir_dot_path} -T png -o output.png'")

console.stderr(f" Generating graph descriptor")

IR graph can be visualized with graphviz command: 'dot webdnn_graph_descriptor/ir.dot -T png -o output.png'
 Generating graph descriptor


In [ ]:
!dot {ir_dot_path} -T png -o output.png

In [14]:
any_backend_failed = False
backends = 'webgpu,webgl,webassembly,fallback'.split(',')
for i, backend in enumerate(backends):
    console.stderr(f"BackendName: {console.colorize(backend, console.Color.Cyan)}")
    try:
        graph_exec_data = generate_descriptor(backend, graph, constant_encoder_name=None)
        graph_exec_data.save(output_dir)
    except Exception as ex:
        if flags.DEBUG:
            raise ex

        any_backend_failed = True
        console.error(f"Failed generating descriptor for {backend} backend")
        console.stderr(traceback.format_exc())
        continue

if any_backend_failed:
    raise ValueError('BackendFailed')
    # raise last_backend_exception

BackendName: webgpu
[WebGPUDescriptorGenerator] Generated kernel source is invalid.
xcrun: error: unable to find utility "metal", not a developer tool or in PATH

Failed generating descriptor for webgpu backend
Traceback (most recent call last):
  File "<ipython-input-14-b76e15d01944>", line 6, in <module>
    graph_exec_data = generate_descriptor(backend, graph, constant_encoder_name=None)
  File "/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/webdnn/backend/interface/generator.py", line 107, in generate_descriptor
    return generator(graph, **kwargs)
  File "/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/webdnn/backend/webgpu/generator.py", line 120, in generate
    return WebGPUDescriptorGenerator.generate(graph, **kwargs)
  File "/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/webdnn/backend/webgpu/generator.py", line 114, in generate
    validate_kernel_source(descriptor)
  File "/Users/mader/anaconda/envs/py36/lib/python3.6/site-packages/web

ValueError: BackendFailed